# Tissue comparison for differential expression analysis

In [ ]:
import functools
import numpy as np
import pandas as pd
from plotnine import *
from scipy.stats import binom_test, fisher_exact, linregress

from warnings import filterwarnings
from matplotlib.cbook import mplDeprecation
filterwarnings('ignore', category=mplDeprecation)
filterwarnings('ignore', category=UserWarning, module='plotnine.*')
filterwarnings('ignore', category=DeprecationWarning, module='plotnine.*')

In [ ]:
config = {
    'caudate': '../../../caudate/_m/genes/diffExpr_EAvsAA_full.txt',
    'dlpfc': '../../../dlpfc/_m/genes/diffExpr_EAvsAA_full.txt',
    'hippo': '../../../hippocampus/_m/genes/diffExpr_EAvsAA_full.txt',
    'gyrus': '../../../dentateGyrus/_m/genes/diffExpr_EAvsAA_full.txt',
}

In [ ]:
@functools.lru_cache()
def get_deg(filename):
    dft = pd.read_csv(filename, sep='\t', index_col=0)
    dft['Feature'] = dft.index
    dft['Dir'] = np.sign(dft['t'])
    if 'gene_id' in dft.columns:
        dft['ensemblID'] = dft.gene_id.str.replace('\\..*', '', regex=True)
    elif 'ensembl_gene_id' in dft.columns:
        dft.rename(columns={'ensembl_gene_id': 'ensemblID'}, inplace=True)
    return dft[['Feature', 'ensemblID', 'adj.P.Val', 'logFC', 't', 'Dir']]

@functools.lru_cache()
def get_deg_sig(filename, fdr):
    dft = get_deg(filename)
    return dft[(dft['adj.P.Val'] < fdr)]


@functools.lru_cache()
def merge_dataframes(tissue1, tissue2):
    return get_deg(config[tissue1]).merge(get_deg(config[tissue2]), 
                                          on='Feature', 
                                          suffixes=['_%s' % tissue1, '_%s' % tissue2])


@functools.lru_cache()
def merge_dataframes_sig(tissue1, tissue2):
    fdr1 = 0.05 if tissue1 != 'dlpfc' else 0.05
    fdr2 = 0.05 if tissue2 != 'dlpfc' else 0.05
    return get_deg_sig(config[tissue1], fdr1).merge(get_deg_sig(config[tissue2], fdr2), 
                                                    on='Feature', 
                                                    suffixes=['_%s' % tissue1, '_%s' % tissue2])

In [ ]:
def enrichment_binom(tissue1, tissue2, merge_fnc):
    df = merge_fnc(tissue1, tissue2)
    df['agree'] = df['Dir_%s' % tissue1] * df['Dir_%s' % tissue2]
    dft = df.groupby('agree').size().reset_index()
    print(dft)
    return binom_test(dft[0].iloc[1], dft[0].sum()) if dft.shape[0] != 1 else print("All directions agree!")


def cal_fishers(tissue1, tissue2):
    df = merge_dataframes(tissue1, tissue2)
    fdr1 = 0.05 if tissue1 != 'dlpfc' else 0.05
    fdr2 = 0.05 if tissue2 != 'dlpfc' else 0.05
    table = [[np.sum((df['adj.P.Val_%s' % tissue1]<fdr1) & 
                     ((df['adj.P.Val_%s' % tissue2]<fdr2))),
              np.sum((df['adj.P.Val_%s' % tissue1]<fdr1) & 
                     ((df['adj.P.Val_%s' % tissue2]>=fdr2)))],
             [np.sum((df['adj.P.Val_%s' % tissue1]>=fdr1) & 
                     ((df['adj.P.Val_%s' % tissue2]<fdr2))),
              np.sum((df['adj.P.Val_%s' % tissue1]>=fdr1) & 
                     ((df['adj.P.Val_%s' % tissue2]>=fdr2)))]]
    print(table)
    return fisher_exact(table)


def calculate_corr(xx, yy):
    '''This calculates R^2 correlation via linear regression:
         - used to calculate relationship between 2 arrays 
         - the arrays are principal components 1 or 2 (PC1, PC2) AND gender
         - calculated on a scale of 0 to 1 (with 0 being no correlation)
        Inputs: 
          x: array of Gender (converted to binary output)
          y: array of PC
        Outputs: 
          1. r2
          2. p-value, two-sided test 
            - whose null hypothesis is that two sets of data are uncorrelated
          3. slope (beta): directory of correlations
    '''
    slope, intercept, r_value, p_value, std_err = linregress(xx, yy)
    return r_value, p_value

    
def corr_annotation(tissue1, tissue2, merge_fnc):
    dft = merge_fnc(tissue1, tissue2)
    xx = dft['t_%s' % tissue1]
    yy = dft['t_%s' % tissue2]
    r_value1, p_value1 = calculate_corr(xx, yy)
    return 'R2: %.2f\nP-value: %.2e' % (r_value1**2, p_value1)


def tissue_annotation(tissue):
    return {'dlpfc': 'DLPFC', 'hippo': 'Hippocampus', 
            'caudate': 'Caudate', 'gyrus': 'Dentate Gyrus'}[tissue]

In [ ]:
def plot_corr_impl(tissue1, tissue2, merge_fnc):
    dft = merge_fnc(tissue1, tissue2)
    title = '\n'.join([corr_annotation(tissue1, tissue2, merge_fnc)])
    xlab = 'T-statistic (%s)' % tissue_annotation(tissue1)
    ylab = 'T-statistic (%s)' % tissue_annotation(tissue2)
    pp = ggplot(dft, aes(x='t_%s'%tissue1, y='t_%s' % tissue2))\
    + geom_point(alpha=0.75, size=3)\
    + theme_matplotlib()\
    + theme(axis_text=element_text(size=18), 
            axis_title=element_text(size=20, face='bold'), 
            plot_title=element_text(size=22))
    pp += labs(x=xlab, y=ylab, title=title)
    return pp


def plot_corr(tissue1, tissue2, merge_fnc):
    return plot_corr_impl(tissue1, tissue2, merge_fnc)


def save_plot(p, fn, width=7, height=7):
    '''Save plot as svg, png, and pdf with specific label and dimension.'''
    for ext in ['.svg', '.png', '.pdf']:
        p.save(fn+ext, width=width, height=height)

## Sample summary

In [ ]:
pheno_file = '../../../../input/phenotypes/merged/_m/merged_phenotypes.csv'
pheno = pd.read_csv(pheno_file, index_col=0)
pheno = pheno[(pheno['Age'] > 17) & 
              (pheno['Dx'].isin(['Schizo', 'Control'])) & 
              (pheno['Race'].isin(['AA', "CAUC"]))].copy()
pheno.head(2)

In [ ]:
pheno.groupby(['Region']).size()

In [ ]:
pheno.groupby(['Region', 'Race']).size()

In [ ]:
pheno.groupby(['Region', 'Race', 'Sex']).size()

## BrainSeq Tissue Comparison

In [ ]:
caudate = get_deg(config['caudate'])
caudate.groupby('Dir').size()

In [ ]:
caudate[(caudate['adj.P.Val'] < 0.05)].shape

In [ ]:
dlpfc = get_deg(config['dlpfc'])
dlpfc.groupby('Dir').size()

In [ ]:
dlpfc[(dlpfc['adj.P.Val'] < 0.05)].shape

In [ ]:
hippo = get_deg(config['hippo'])
hippo.groupby('Dir').size()

In [ ]:
hippo[(hippo['adj.P.Val'] < 0.05)].shape

In [ ]:
gyrus = get_deg(config['gyrus'])
gyrus.groupby('Dir').size()

In [ ]:
gyrus[(gyrus['adj.P.Val'] < 0.05)].shape

### Enrichment of DEG

In [ ]:
cal_fishers('caudate', 'dlpfc')

In [ ]:
cal_fishers('caudate', 'hippo')

In [ ]:
cal_fishers('dlpfc', 'hippo')

In [ ]:
cal_fishers('caudate', 'gyrus')

In [ ]:
cal_fishers('dlpfc', 'gyrus')

In [ ]:
cal_fishers('hippo', 'gyrus')

### Correlation

In [ ]:
pp = plot_corr('caudate', 'dlpfc', merge_dataframes)
pp

In [ ]:
qq = plot_corr('caudate', 'hippo', merge_dataframes)
qq

In [ ]:
ww = plot_corr('dlpfc', 'hippo', merge_dataframes)
ww

In [ ]:
rr = plot_corr('caudate', 'gyrus', merge_dataframes)
rr

In [ ]:
ss = plot_corr('dlpfc', 'gyrus', merge_dataframes)
ss

In [ ]:
tt = plot_corr('hippo', 'gyrus', merge_dataframes)
tt

### Significant correlation, FDR < 0.05

In [ ]:
pp = plot_corr('caudate', 'dlpfc', merge_dataframes_sig)
pp

In [ ]:
qq = plot_corr('caudate', 'hippo', merge_dataframes_sig)
qq

In [ ]:
ww = plot_corr('dlpfc', 'hippo', merge_dataframes_sig)
ww

In [ ]:
rr = plot_corr('caudate', 'gyrus', merge_dataframes_sig)
rr

In [ ]:
ss = plot_corr('dlpfc', 'gyrus', merge_dataframes_sig)
ss

In [ ]:
tt = plot_corr('hippo', 'gyrus', merge_dataframes_sig)
tt

In [ ]:
#save_plot(pp, 'dlpfc_caudate_tstatistic_corr_sig')
#save_plot(qq, 'hippo_caudate_tstatistic_corr_sig')
#save_plot(ww, 'hippo_dlpfc_tstatistic_corr_sig')

### Directionality test

#### All genes

In [ ]:
enrichment_binom('caudate', 'dlpfc', merge_dataframes)

In [ ]:
enrichment_binom('caudate', 'hippo', merge_dataframes)

In [ ]:
enrichment_binom('dlpfc', 'hippo', merge_dataframes)

In [ ]:
enrichment_binom('caudate', 'gyrus', merge_dataframes)

In [ ]:
enrichment_binom('dlpfc', 'gyrus', merge_dataframes)

In [ ]:
enrichment_binom('hippo', 'gyrus', merge_dataframes)

#### Significant DEG (FDR < 0.05)

In [ ]:
enrichment_binom('caudate', 'dlpfc', merge_dataframes_sig)

In [ ]:
enrichment_binom('caudate', 'hippo', merge_dataframes_sig)

In [ ]:
enrichment_binom('dlpfc', 'hippo', merge_dataframes_sig)

In [ ]:
enrichment_binom('caudate', 'gyrus', merge_dataframes_sig)

In [ ]:
enrichment_binom('dlpfc', 'gyrus', merge_dataframes_sig)

In [ ]:
enrichment_binom('hippo', 'gyrus', merge_dataframes_sig)